# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [2]:
import pandas as pd

---
## 2. Load data

In [5]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [7]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [10]:
price_df = pd.read_csv('price_data.csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [12]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as yo can!

In [15]:
df.dtypes

id                                        object
channel_sales                             object
cons_12m                                   int64
cons_gas_12m                               int64
cons_last_month                            int64
date_activ                        datetime64[ns]
date_end                          datetime64[ns]
date_modif_prod                   datetime64[ns]
date_renewal                      datetime64[ns]
forecast_cons_12m                        float64
forecast_cons_year                         int64
forecast_discount_energy                 float64
forecast_meter_rent_12m                  float64
forecast_price_energy_off_peak           float64
forecast_price_energy_peak               float64
forecast_price_pow_off_peak              float64
has_gas                                   object
imp_cons                                 float64
margin_gross_pow_ele                     float64
margin_net_pow_ele                       float64
nb_prod_act         

# 🔍 Understanding Features in `clean_data_after_eda.csv`

This table documents each feature, what it represents, and whether it is potentially useful for churn prediction. This understanding will guide our feature engineering process.

| Column Name                     | Description                                                            | Type         | Useful for Churn? | Notes                                                                 |
|--------------------------------|------------------------------------------------------------------------|--------------|-------------------|-----------------------------------------------------------------------|
| id                             | Customer's unique identifier                                           | object       | ❌ No              | Only used for joins, not modeling                                     |
| channel_sales                  | Distribution channel (e.g., third-party, online)                       | object       | ✅ Yes             | We saw variation in churn across channels                            |
| cons_12m                       | Electricity consumption over past 12 months                            | int64        | ✅ Yes             | Can be useful in trend/rate comparisons                              |
| cons_gas_12m                   | Gas consumption over past 12 months                                    | int64        | ✅ Yes             | May help with bundling insights or customer type                     |
| cons_last_month                | Electricity consumption in last month                                  | int64        | ✅ Yes             | Can signal declining usage before churn                              |
| date_activ                     | Contract activation date                                               | datetime     | 🟡 Indirect        | Needed to compute age/tenure                                          |
| date_end                       | Contract end date                                                      | datetime     | 🟡 Indirect        | May signal nearing renewal                                           |
| date_modif_prod                | Date product was modified                                              | datetime     | 🟡 Indirect        | Could be used to track recency of changes                            |
| date_renewal                   | Renewal date                                                           | datetime     | 🟡 Indirect        | Could calculate time since last renewal                             |
| forecast_cons_12m             | Forecasted electricity consumption                                     | float64      | ✅ Yes             | Compare to actuals for trust/accuracy gaps                           |
| forecast_cons_year            | Forecasted yearly consumption                                          | int64        | 🟡 Marginal        | Might be redundant with 12m version                                  |
| forecast_discount_energy      | Forecasted energy discount                                             | float64      | 🟡 Maybe           | Mostly zero — likely not predictive                                  |
| forecast_meter_rent_12m       | Forecasted meter rental fee                                            | float64      | ✅ Yes             | Higher fixed fees → more churn risk                                  |
| forecast_price_energy_off_peak| Forecasted off-peak energy price                                       | float64      | 🟡 Maybe           | Low variation — low signal                                           |
| forecast_price_energy_peak    | Forecasted peak energy price                                           | float64      | 🟡 Maybe           | Same as above                                                        |
| forecast_price_pow_off_peak   | Forecasted off-peak power price                                        | float64      | 🟡 Maybe           | Same as above                                                        |
| has_gas                       | Whether customer has gas service                                       | object       | ✅ Yes             | Bundled customers less likely to churn                              |
| imp_cons                      | Imputed consumption (estimation)                                       | float64      | 🟡 Maybe           | May be useful for engineered volatility                             |
| margin_gross_pow_ele          | Gross electricity profit margin                                        | float64      | ✅ Yes             | High-margin customers may churn more if dissatisfied                |
| margin_net_pow_ele            | Net electricity profit margin                                          | float64      | ✅ Yes             | Similar reason as above                                              |
| nb_prod_act                   | Number of active products                                              | int64        | ✅ Yes             | Bundling reduces churn                                               |
| net_margin                    | Overall net profit                                                     | float64      | ✅ Yes             | Important business value variable                                   |
| num_years_antig               | Years as customer                                                      | int64        | ✅ Yes             | Tenure may affect churn                                              |
| origin_up                     | Customer acquisition source                                            | object       | ✅ Yes             | Acquisition source affects stickiness                               |
| pow_max                       | Maximum contracted power                                               | float64      | ✅ Yes             | High capacity customers may be more sensitive to pricing            |
| var_year_* and var_6m_*       | Yearly and 6-month price variance (peak/off-peak/fix/var)              | float64      | ✅ Yes             | Volatility can signal price shocks → churn                          |
| churn                         | Target variable                                                        | int64        | 🎯 Target          | We aim to predict this                                               |

🟡 = potentially useful for feature engineering, not direct input


### 🔧 Feature Engineering Plan: 10 New Features

| #  | Feature Name                | Type       | Core Columns Used                            | Why It Matters                                          |
|----|-----------------------------|------------|-----------------------------------------------|----------------------------------------------------------|
| 1  | `account_age_days`          | Numerical  | `date_activ`                                  | Older customers may be stickier                         |
| 2  | `margin_per_product`        | Numerical  | `net_margin`, `nb_prod_act`                   | Per-product profitability                               |
| 3  | `forecast_error_ratio`      | Numerical  | `forecast_cons_12m`, `cons_12m`               | How wrong the forecast is                               |
| 4  | `cost_per_kwh`              | Numerical  | `forecast_meter_rent_12m`, `forecast_cons_12m`| Effective cost burden                                   |
| 5  | `consumption_change_ratio`  | Numerical  | `cons_last_month`, `cons_12m`                 | Drop-off in usage can signal churn                      |
| 6  | `price_volatility_6m_total` | Numerical  | All `var_6m_price_*` columns                  | Sum of short-term pricing volatility                    |
| 7  | `has_gas_binary`            | Categorical| `has_gas`                                     | Binary gas service indicator                            |
| 8  | `channel_grouped`           | Categorical| `channel_sales`                               | Group low-volume or zero-churn codes                    |
| 9  | `net_to_gross_margin_ratio` | Numerical  | `margin_net_pow_ele`, `margin_gross_pow_ele` | Internal margin analysis                                |
| 10 | `power_margin_efficiency`   | Numerical  | `net_margin`, `pow_max`                       | Profit generated per unit of power                      |


#### We will be engineering 8 numerical features and 2 well thought out categorical features

### Feature 1: `account_age_days`
This feature represents the number of day since the customer activated their account.  It's derived from the `date_activ` field.  Older customers may be less likely to churn due to sunk costs, brand familiarity, or bundled offers over time.

In [21]:
from datetime import datetime

In [23]:
df['date_activ'] = pd.to_datetime(df['date_activ'])# convert dates to datetime format for easy operations later
df['account_age_days'] = (pd.to_datetime("today") - df['date_activ']).dt.days # subtracting date active from today's date 
#gives account age in days for each row or each client.
df['account_age_days'].describe() #gives us descriptive statistics of our new row/feature

count    14606.000000
mean      5231.670615
std        589.317924
min       3920.000000
25%       4785.000000
50%       5197.000000
75%       5610.000000
max       8053.000000
Name: account_age_days, dtype: float64

In [25]:
df[['account_age_days']].head(8)

,account_age_days
0,4363
1,5757
2,5519
3,5536
4,5612
5,4917
6,4924
7,5514


### Feature 2: `margin_per_product`
This feature represents the customer's profitability per active product.  It is calculated by dividing the `net_margin` by `nb_prod_act`. A higher value could indicate a highly profitable customer who might be more sensitive to pricing changes or overpaying for fewer services.  This feature may help identify customers at risk of churn due to low perceived value.

In [28]:
df['margin_per_product'] = df['net_margin'] / (df['nb_prod_act'] + 1)
#add 1 to nb_prod_act to avoid divide by 0 error
df[['net_margin', 'nb_prod_act', 'margin_per_product']].head()

,net_margin,nb_prod_act,margin_per_product
0,678.99,2,226.330
1,18.89,1,9.445
2,6.60,1,3.300
3,25.46,1,12.730
4,47.98,1,23.990


### Feature 3: `forecast_error_ratio`

This feature captures the percentage error between forecasted consumption and actual consumption over the last 12 months.  The large forecast error may signal dissatisfaction, billing disputes, or poor usage understanding - all of which can increase the likelihood of the client churning.

Formula: forecast_error_ratio = (forecast_cons_12m - cons_12m) / (cons_12m + 1)

We add 1 to the denominator to avoid division_by-zero errors.

In [31]:
df['forecast_error_ratio'] = (df['forecast_cons_12m'] - df['cons_12m']) / (df['cons_12m'] + 1)
df[['forecast_cons_12m', 'cons_12m', 'forecast_error_ratio']].head()

,forecast_cons_12m,cons_12m,forecast_error_ratio
0,0.00,0,0.000000
1,189.95,4660,-0.959032
2,47.96,544,-0.910165
3,240.04,1584,-0.847924
4,445.75,4425,-0.899062


##### Negative values means the customer used more than what was forecasted(underforecasting)
##### Positive values means they used less than what's forecasted(overforecasting)
##### Wheather positive or negative, the higher the absolute error ratio, the higher the risk or client churn

### Feature 4: `cost_per_kwh`

This feature estimates the customer's cost per kilowatt-hour(kWh) by dividing their forecasted meter rent over 12 months(euros) by their forecasted electricity consumption over 12 months(kilowatts per hour).  It helps us understand each customer's cost-efficiency.  A high `cost_per_kwh` may indicate a higher perceived cost and therefore a higher risk of churn.

Formula: cost_per_kwh = forecast_meter_rent_12m / (forecast_cons_12m + 1) 

In [35]:
df['cost_per_kwh'] = df['forecast_meter_rent_12m'] / (df['forecast_cons_12m'] + 1)
df[['forecast_meter_rent_12m',  'forecast_cons_12m', 'cost_per_kwh']].head()

,forecast_meter_rent_12m,forecast_cons_12m,cost_per_kwh
0,1.78,0.00,1.780000
1,16.27,189.95,0.085206
2,38.72,47.96,0.790850
3,19.83,240.04,0.082269
4,131.73,445.75,0.294863


### Feature 5: `consumption_change_ratio`

This feature compares recent electricity consumption to long-term usage. Specifically, it divides `cons_last_month` by the average monthly consumption over the past 12 months(`cons_12m / 12`).

Customers whose recent consumption is much lower than their long-term average may be:
-Reducing usage
-Switching to competitors
-Moving out or scaling down

Smaller the fraction or decimal means higher risk of churn
These changes can signal a higher risk of churn.

Formula:
consumption_change_ratio = cons_last_month / ((cons_12m/12) + 1)

In [38]:
df['consumption_change_ratio'] = df['cons_last_month'] / ((df['cons_12m'] / 12) + 1)
df[['cons_last_month', 'cons_12m', 'consumption_change_ratio']].head()

,cons_last_month,cons_12m,consumption_change_ratio
0,0,0,0.000000
1,0,4660,0.000000
2,0,544,0.000000
3,0,1584,0.000000
4,526,4425,1.422583


### Feature 6: `price_volatility_6m_total`

This feature sums all 6-month price variance columns (across peak, mid-peak, and off-peak; fixed and variable) into one single volatility score.  High price volatility can frustrate clients, making them more likely to churn.

We're summing the following columns:
-`var_6m_price_off_peak_var`
-`var_6m_price_peak_var`
-`var_6m_price_mid_peak_var`
-`var_6m_price_off_peak_fix`
-`var_6m_price_peak_fix`
-`var_6m_price_mid_peak_fix`

This gives a composite measure of how erratic pricing has been over the past 6 months.

Formula: 

price_volatility_6m_total = sum of all 6-month variance columns

In [41]:
volatility_cols = ['var_6m_price_off_peak_var',
    'var_6m_price_peak_var',
    'var_6m_price_mid_peak_var',
    'var_6m_price_off_peak_fix',
    'var_6m_price_peak_fix',
    'var_6m_price_mid_peak_fix']

df['price_volatility_6m_total'] = df[volatility_cols].sum(axis = 1)

df[volatility_cols + ['price_volatility_6m_total']].head()

,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,price_volatility_6m_total
0,0.000131,4.100838e-05,9.084737e-04,2.086294,99.530517,44.235794,145.853685
1,0.000003,1.217891e-03,0.000000e+00,0.009482,0.000000,0.000000,0.010703
2,0.000004,9.450150e-08,0.000000e+00,0.000000,0.000000,0.000000,0.000004
3,0.000003,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000003
4,0.000011,2.896760e-06,4.860000e-10,0.000000,0.000000,0.000000,0.000014


### Feature 7: `has_gas_binary`

This is a binary encoding of the `has_gas` column, which indicates whether a customer has an active gas service.

Customers with bundled utility services (both electricity and gas) are often less likely to churn
-Bundles increase perceived value
-Leaving one service means leaving both
-Loyalty programs often reward bundled plans

We convert:

- `'t'` (true) : 1
- `'f'` (false) : 0

In [44]:
df['has_gas_binary'] = df['has_gas'].map({'t': 1, 'f': 0})

df[['has_gas', 'has_gas_binary']].head()

,has_gas,has_gas_binary
0,t,1
1,f,0
2,f,0
3,f,0
4,f,0


##### This encoding will be useful when we train our model.

### Feature 8: `channel_grouped`

The original `channel_sales` column contains many cryptic categorical codes. Some of them have very few observations or produce zero churn, while others correlate strongly with churn.

This feature simplifies the channel space by grouping:
- The top 3 most common channels by frequency into their own categories
- Everything else into a single `Other` category

This reduces noise and improves model generalization.

Steps:
1. Count value frequencies
2. Retain the top 3 most common codes
3. Replace the rest with `Other`

In [48]:
top_channels = df['channel_sales'].value_counts().nlargest(3).index.tolist()

channel_mapping = {
    top_channels[0]: 1,
    top_channels[1]: 2,
    top_channels[2]: 3
}

df['channel_grouped'] = df['channel_sales'].apply(lambda x: channel_mapping[x] if x in channel_mapping else 0)

print("Mapping:")
for k, v in channel_mapping.items():
    print(f"{v} : {k}")

df[['channel_sales', 'channel_grouped']].head(10)

Mapping:
1 : foosdfpfkusacimwkcsosbicdxkicaua
2 : MISSING
3 : lmkebamcaaclubfxadlmueccxoimlema


,channel_sales,channel_grouped
0,foosdfpfkusacimwkcsosbicdxkicaua,1
1,MISSING,2
2,foosdfpfkusacimwkcsosbicdxkicaua,1
3,lmkebamcaaclubfxadlmueccxoimlema,3
4,MISSING,2
5,usilxuppasemubllopkaafesmlibmsdf,0
6,foosdfpfkusacimwkcsosbicdxkicaua,1
7,foosdfpfkusacimwkcsosbicdxkicaua,1
8,usilxuppasemubllopkaafesmlibmsdf,0
9,lmkebamcaaclubfxadlmueccxoimlema,3


### Feature 9: `net_to_gross_margin_ratio`

This feature represents the efficiency of converting gross electricity margin to net margin.  It helps asses profitability while factoring in operating or distribution costs.

Low ratios may suggest high overheads of poor pricing efficiency - possibly frustrating customers and learing to churn.

Formula:
net_to_gross_margin_ratio = margin_net_pow_ele / (margin_gross_pow_ele + 1)

In [51]:
df['net_to_gross_margin_ratio'] = df['margin_net_pow_ele'] / (df['margin_gross_pow_ele'] + 1)

df[['margin_net_pow_ele', 'margin_gross_pow_ele', 'net_to_gross_margin_ratio']].head()

,margin_net_pow_ele,margin_gross_pow_ele,net_to_gross_margin_ratio
0,25.44,25.44,0.962179
1,16.38,16.38,0.942463
2,28.60,28.60,0.966216
3,30.22,30.22,0.967969
4,44.91,44.91,0.978218


### Feature 10: `pwer_margin_efficiency`

This feature evaluates how efficiently a client converts their subscribed power capacity into net profit for their company.  It's especially helpful when power capacity is loosely tied to consumption.

A customer with high power capacity but low margin could signal:
- Inefficient pricing
- Underutilization
- High churn risk due to perceived poor value

Smaller the ratio or fraction or decimal value, the higher the risk of churn since it indicates that this client may be feeling like they are overpaying for underutilized capacity.

Formula:
power_margin_efficiency = net_margin / (pow_max + 1)

In [54]:
df['power_margin_efficiency'] = df['net_margin'] / (df['pow_max'] + 1)

df[['net_margin', 'pow_max', 'power_margin_efficiency']].head()

,net_margin,pow_max,power_margin_efficiency
0,678.99,43.648,15.207624
1,18.89,13.800,1.276351
2,6.60,13.856,0.444265
3,25.46,13.200,1.792958
4,47.98,19.800,2.306731


In [56]:
df.columns

Index(['id', 'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month',
       'date_activ', 'date_end', 'date_modif_prod', 'date_renewal',
       'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'origin_up', 'pow_max',
       'var_year_price_off_peak_var', 'var_year_price_peak_var',
       'var_year_price_mid_peak_var', 'var_year_price_off_peak_fix',
       'var_year_price_peak_fix', 'var_year_price_mid_peak_fix',
       'var_year_price_off_peak', 'var_year_price_peak',
       'var_year_price_mid_peak', 'var_6m_price_off_peak_var',
       'var_6m_price_peak_var', 'var_6m_price_mid_peak_var',
       'var_6m_price_off_peak_fix', 'var_6m_price_peak_fix',
       'var_6m_price_mid_peak_fix', 'var_6m_p

In [58]:
df = df.merge(diff[['id', 'offpeak_diff_dec_january_energy', 'offpeak_diff_dec_january_power']],
    on = 'id', how = 'left'
)

In [60]:
df.to_csv("mid_correction_data.csv", index = False)

In [62]:
df.columns

Index(['id', 'channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month',
       'date_activ', 'date_end', 'date_modif_prod', 'date_renewal',
       'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas',
       'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'origin_up', 'pow_max',
       'var_year_price_off_peak_var', 'var_year_price_peak_var',
       'var_year_price_mid_peak_var', 'var_year_price_off_peak_fix',
       'var_year_price_peak_fix', 'var_year_price_mid_peak_fix',
       'var_year_price_off_peak', 'var_year_price_peak',
       'var_year_price_mid_peak', 'var_6m_price_off_peak_var',
       'var_6m_price_peak_var', 'var_6m_price_mid_peak_var',
       'var_6m_price_off_peak_fix', 'var_6m_price_peak_fix',
       'var_6m_price_mid_peak_fix', 'var_6m_p

### Now that we made 11 new features, we will go through the rest of our updated dataset and build a pipeline to fully clean our dataset for modeling in the next step.

In [65]:
final_df = pd.read_csv("mid_correction_data.csv")

In [67]:
final_df.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,forecast_error_ratio,cost_per_kwh,consumption_change_ratio,price_volatility_6m_total,has_gas_binary,channel_grouped,net_to_gross_margin_ratio,power_margin_efficiency,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000000,1.780000,0.000000,145.853685,1,1,0.962179,15.207624,0.020057,3.700961
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,-0.959032,0.085206,0.000000,0.010703,0,2,0.942463,1.276351,-0.003767,0.177779
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,-0.910165,0.790850,0.000000,0.000004,0,1,0.966216,0.444265,-0.004670,0.177779
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,-0.847924,0.082269,0.000000,0.000003,0,3,0.967969,1.792958,-0.004547,0.177779
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,-0.899062,0.294863,1.422583,0.000014,0,2,0.978218,2.306731,-0.006192,0.162916


In [69]:
#Drop raw data we already encoded into useful features
final_df.drop(columns = ['channel_sales', 'has_gas', 'date_activ', 'date_end', 'date_modif_prod', 'date_renewal'], inplace = True, errors = 'ignore')

In [71]:
#One-hot encode 'origin_up' into dummy variables

final_df = pd.get_dummies(final_df, columns = ['origin_up'], drop_first = True)

In [73]:
#Log Transform skewed numerical columns

In [75]:
import numpy as np
numeric_cols = final_df.select_dtypes(include = 'number').columns
excluded = ['churn'] + [col for col in final_df.columns if 'channel_grouped_' in col or 'origin_up' in col]
log_candidates = [col for col in numeric_cols if col not in excluded and (final_df[col] > 0).all()]
for col in log_candidates:
    final_df[f'log_{col}'] = np.log(final_df[col] + 1)

In [77]:
#drop id column too much noise
final_df.drop(columns = 'id', inplace = True, errors = 'ignore')

In [79]:
final_df.head()

,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,...,offpeak_diff_dec_january_power,origin_up_ewxeelcelemmiwuafmddpobolfuxioce,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw,origin_up_usapbepcfoloekilkwsdiboslwaxobdp,log_nb_prod_act,log_num_years_antig,log_pow_max,log_account_age_days
0,0,54946,0,0.00,0,0.0,1.78,0.114481,0.098142,40.606701,...,3.700961,False,False,False,True,False,1.098612,1.386294,3.798810,8.381144
1,4660,0,0,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,...,0.177779,False,True,False,False,False,0.693147,1.945910,2.694627,8.658345
2,544,0,0,47.96,0,0.0,38.72,0.165794,0.087899,44.311378,...,0.177779,False,True,False,False,False,0.693147,1.945910,2.698404,8.616133
3,1584,0,0,240.04,0,0.0,19.83,0.146694,0.000000,44.311378,...,0.177779,False,True,False,False,False,0.693147,1.945910,2.653242,8.619208
4,4425,0,526,445.75,526,0.0,131.73,0.116900,0.100015,40.606701,...,0.162916,False,True,False,False,False,0.693147,1.945910,3.034953,8.632841


In [81]:
final_df.columns

Index(['cons_12m', 'cons_gas_12m', 'cons_last_month', 'forecast_cons_12m',
       'forecast_cons_year', 'forecast_discount_energy',
       'forecast_meter_rent_12m', 'forecast_price_energy_off_peak',
       'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'imp_cons',
       'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act',
       'net_margin', 'num_years_antig', 'pow_max',
       'var_year_price_off_peak_var', 'var_year_price_peak_var',
       'var_year_price_mid_peak_var', 'var_year_price_off_peak_fix',
       'var_year_price_peak_fix', 'var_year_price_mid_peak_fix',
       'var_year_price_off_peak', 'var_year_price_peak',
       'var_year_price_mid_peak', 'var_6m_price_off_peak_var',
       'var_6m_price_peak_var', 'var_6m_price_mid_peak_var',
       'var_6m_price_off_peak_fix', 'var_6m_price_peak_fix',
       'var_6m_price_mid_peak_fix', 'var_6m_price_off_peak',
       'var_6m_price_peak', 'var_6m_price_mid_peak', 'churn',
       'account_age_days', 'margin_pe

In [83]:
#drop low variance and few unique values but keep binary variables
low_variance_cols = []

numerical_df = final_df.select_dtypes(include = 'number')

for col in numerical_df.columns:
    unique_vals = numerical_df[col].nunique()
    if unique_vals <= 2:
        continue #keeping binaries
    variance = numerical_df[col].var()
    if variance < 0.01 or unique_vals < 5:
        low_variance_cols.append(col)

print(low_variance_cols)

['forecast_price_energy_off_peak', 'forecast_price_energy_peak', 'var_year_price_off_peak_var', 'var_year_price_peak_var', 'var_year_price_mid_peak_var', 'var_6m_price_off_peak_var', 'var_6m_price_peak_var', 'var_6m_price_mid_peak_var', 'channel_grouped', 'offpeak_diff_dec_january_energy']


In [85]:
final_low_variance_cols = [
    'forecast_price_energy_off_peak',
    'forecast_price_energy_peak',
    'var_year_price_off_peak_var',
    'var_year_price_peak_var',
    'var_year_price_mid_peak_var',
    'var_6m_price_off_peak_var',
    'var_6m_price_peak_var',
    'var_6m_price_mid_peak_var'
]
final_df.drop(columns=final_low_variance_cols, inplace=True)

In [87]:
#Show correlation matrix
correlation_matrix = final_df.corr(numeric_only = True).round(2)
correlation_matrix

,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_pow_off_peak,imp_cons,margin_gross_pow_ele,...,offpeak_diff_dec_january_power,origin_up_ewxeelcelemmiwuafmddpobolfuxioce,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw,origin_up_usapbepcfoloekilkwsdiboslwaxobdp,log_nb_prod_act,log_num_years_antig,log_pow_max,log_account_age_days
cons_12m,1.00,0.49,0.97,0.19,0.17,-0.04,0.07,-0.03,0.16,-0.01,...,-0.02,-0.00,0.09,-0.03,-0.05,-0.00,0.15,0.01,0.09,-0.01
cons_gas_12m,0.49,1.00,0.51,0.08,0.08,-0.01,0.04,-0.02,0.08,0.01,...,-0.01,-0.00,0.04,-0.02,-0.02,-0.00,0.27,-0.01,0.06,-0.02
cons_last_month,0.97,0.51,1.00,0.18,0.19,-0.04,0.06,-0.02,0.19,-0.01,...,-0.02,-0.00,0.08,-0.03,-0.05,-0.00,0.16,0.01,0.08,-0.01
forecast_cons_12m,0.19,0.08,0.18,1.00,0.65,0.06,0.31,-0.02,0.63,-0.02,...,-0.07,-0.00,-0.07,0.03,0.04,-0.00,0.07,-0.00,0.39,0.00
forecast_cons_year,0.17,0.08,0.19,0.65,1.00,0.01,0.28,-0.04,0.97,-0.01,...,-0.04,-0.00,-0.04,0.03,0.01,0.00,0.05,0.03,0.31,0.03
forecast_discount_energy,-0.04,-0.01,-0.04,0.06,0.01,1.00,-0.01,0.05,0.04,0.24,...,0.08,-0.00,-0.06,-0.03,0.08,-0.00,0.13,-0.07,-0.00,-0.07
forecast_meter_rent_12m,0.07,0.04,0.06,0.31,0.28,-0.01,1.00,-0.20,0.22,0.18,...,-0.07,-0.01,-0.11,0.06,0.05,-0.01,0.02,0.04,0.74,0.06
forecast_price_pow_off_peak,-0.03,-0.02,-0.02,-0.02,-0.04,0.05,-0.20,1.00,-0.01,-0.06,...,0.14,0.00,0.06,-0.01,-0.06,0.01,-0.01,-0.10,-0.22,-0.11
imp_cons,0.16,0.08,0.19,0.63,0.97,0.04,0.22,-0.01,1.00,-0.00,...,-0.04,-0.00,-0.03,0.02,0.01,0.00,0.05,0.01,0.26,0.01
margin_gross_pow_ele,-0.01,0.01,-0.01,-0.02,-0.01,0.24,0.18,-0.06,-0.00,1.00,...,0.15,-0.00,-0.04,-0.06,0.08,-0.01,-0.00,-0.07,0.38,-0.09


In [89]:
# convert all boolean columns in the DataFrame to 1 or 0
bool_cols = final_df.select_dtypes(include='bool').columns
final_df[bool_cols] = final_df[bool_cols].astype(int)

In [91]:
final_df.head()

,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_pow_off_peak,imp_cons,margin_gross_pow_ele,...,offpeak_diff_dec_january_power,origin_up_ewxeelcelemmiwuafmddpobolfuxioce,origin_up_kamkkxfxxuwbdslkwifmmcsiusiuosws,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw,origin_up_usapbepcfoloekilkwsdiboslwaxobdp,log_nb_prod_act,log_num_years_antig,log_pow_max,log_account_age_days
0,0,54946,0,0.00,0,0.0,1.78,40.606701,0.00,25.44,...,3.700961,0,0,0,1,0,1.098612,1.386294,3.798810,8.381144
1,4660,0,0,189.95,0,0.0,16.27,44.311378,0.00,16.38,...,0.177779,0,1,0,0,0,0.693147,1.945910,2.694627,8.658345
2,544,0,0,47.96,0,0.0,38.72,44.311378,0.00,28.60,...,0.177779,0,1,0,0,0,0.693147,1.945910,2.698404,8.616133
3,1584,0,0,240.04,0,0.0,19.83,44.311378,0.00,30.22,...,0.177779,0,1,0,0,0,0.693147,1.945910,2.653242,8.619208
4,4425,0,526,445.75,526,0.0,131.73,40.606701,52.32,44.91,...,0.162916,0,1,0,0,0,0.693147,1.945910,3.034953,8.632841


In [93]:
final_df.to_csv("finalized_cleaned_data.csv", index = False)